In [ ]:
# =============================================================================
# TCN multistep (covariables)  –  incluye codificadores cíclicos
# =============================================================================
import os, random, numpy as np, pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks

# ------------------------- reproducibilidad ----------------------------------
SEED = 28371
os.environ["PYTHONHASHSEED"] = str(SEED)
np.random.seed(SEED)
random.seed(SEED)
tf.keras.utils.set_random_seed(SEED)
tf.config.experimental.enable_op_determinism()

# ------------------------- parámetros ----------------------------------------
BASE_FEATURES = ["dd_Valor", "VRB_Valor", "Presion_QFE",
                 "Temperatura", "Presion_QFF"]       # las 5 a pronosticar
CYCLIC_FEATS  = ["hora_sin", "hora_cos", "doy_sin", "doy_cos"]
FEATURES      = BASE_FEATURES + CYCLIC_FEATS          # total 9 columnas

WINDOW   = 28 * 24      # 28 días de contexto
HORIZON  = 30 * 24     # 720 horas
TEST_RATIO = 0.2

# ------------------------- 1) datos ------------------------------------------
df = base_estacion_290004.copy()
df["momento"] = pd.to_datetime(df["momento"], dayfirst=True)

# ---- variables cíclicas -----------------------------------------------------
df["hora"] = df["momento"].dt.hour
df["doy"]  = df["momento"].dt.dayofyear          # 1…365 (o 366)

df["hora_sin"] = np.sin(2*np.pi*df["hora"] / 24)
df["hora_cos"] = np.cos(2*np.pi*df["hora"] / 24)

df["doy_sin"]  = np.sin(2*np.pi*df["doy"] / 365)
df["doy_cos"]  = np.cos(2*np.pi*df["doy"] / 365)

# -----------------------------------------------------------------------------
CUTOFF = pd.Timestamp("2025-06-23 08:00")
df_hist = df[df["momento"] <= CUTOFF].copy()

X_full = df_hist[FEATURES].values              # (n, 9)  ← incluye senos/cosenos
Y_full = df_hist[BASE_FEATURES].values         # (n, 5)  ← solo las físicas

# ------------------------- 2) ventanas ---------------------------------------
def make_seq(X, Y, win, hor):
    xs, ys = [], []
    last = len(X) - win - hor + 1
    for i in range(last):
        xs.append(X[i : i+win])
        ys.append(Y[i+win : i+win+hor])
    return np.asarray(xs), np.asarray(ys)

X_raw, Y_raw = make_seq(X_full, Y_full, WINDOW, HORIZON)

split = int(len(X_raw) * (1 - TEST_RATIO))
X_train_raw, X_test_raw = X_raw[:split], X_raw[split:]
Y_train_raw, Y_test_raw = Y_raw[:split], Y_raw[split:]

# ------------------------- 3) escalado ---------------------------------------
x_scaler, y_scaler = StandardScaler(), StandardScaler()
x_scaler.fit(X_train_raw.reshape(-1, len(FEATURES)))
y_scaler.fit(Y_train_raw.reshape(-1, len(BASE_FEATURES)))

def scale(Xr, Yr):
    Xs = x_scaler.transform(Xr.reshape(-1, len(FEATURES))).reshape(Xr.shape)
    Ys = y_scaler.transform(Yr.reshape(-1, len(BASE_FEATURES))).reshape(Yr.shape)
    return Xs.astype("float32"), Ys.astype("float32")

X_train, Y_train = scale(X_train_raw, Y_train_raw)
X_test,  Y_test  = scale(X_test_raw,  Y_test_raw)

# ------------------------- 4) modelo TCN -------------------------------------
def tcn_block(x, filters, k, d, seed):
    prev = x
    for i in range(2):
        x = layers.Conv1D(filters, k, padding="causal",
                          dilation_rate=d, activation="relu",
                          kernel_initializer=tf.keras.initializers.GlorotUniform(seed+i))(x)
        x = layers.Dropout(0.2, seed=seed+i)(x)
    if prev.shape[-1] != filters:
        prev = layers.Conv1D(filters, 1, padding="same",
                             kernel_initializer=tf.keras.initializers.GlorotUniform(seed))(prev)
    return layers.Activation("relu")(layers.Add()([x, prev]))

inputs = layers.Input(shape=(WINDOW, len(FEATURES)))
x = tcn_block(inputs, 32, 3, 1, SEED)
x = tcn_block(x,      32, 3, 2, SEED+10)
x = tcn_block(x,      32, 3, 4, SEED+20)
x = tcn_block(x,      32, 3, 8, SEED+30)
x = tcn_block(x,      32, 3, 16, SEED+40)
x = tcn_block(x,      32, 3, 32, SEED+50)
x = tcn_block(x,      32, 3, 64, SEED+60)
x = layers.Flatten()(x)
flat_out = layers.Dense(HORIZON * len(BASE_FEATURES))(x)
outputs  = layers.Reshape((HORIZON, len(BASE_FEATURES)))(flat_out)

model = models.Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss=tf.keras.losses.Huber(delta=0.25))
model.summary()

# ------------------------- 5) entrenamiento ----------------------------------
history = model.fit(
    X_train, Y_train,
    validation_data=(X_test, Y_test),
    epochs=150, batch_size=32, shuffle=False,
    callbacks=[callbacks.EarlyStopping("val_loss", patience=10,
                                       restore_best_weights=True)],
    verbose=2
)

# ------------------------- 6) pronóstico 720 h -------------------------------
last_window_df = df_hist.tail(WINDOW)
X_last = x_scaler.transform(last_window_df[FEATURES]).reshape(1, WINDOW, len(FEATURES))

pred_scaled = model.predict(X_last)[0]                        # (720, 5)
pred        = y_scaler.inverse_transform(pred_scaled)

# --- genera las variables cíclicas (deterministas) para el horizonte ----------
future_idx = pd.date_range(start=CUTOFF + pd.Timedelta(hours=1),
                           periods=HORIZON, freq="H")

cyclic_future = pd.DataFrame({
    "hora_sin": np.sin(2*np.pi*future_idx.hour / 24),
    "hora_cos": np.cos(2*np.pi*future_idx.hour / 24),
    "doy_sin":  np.sin(2*np.pi*future_idx.dayofyear / 365),
    "doy_cos":  np.cos(2*np.pi*future_idx.dayofyear / 365),
}, index=future_idx)

df_future_feats = pd.DataFrame(pred, columns=[f"{c}_pred" for c in BASE_FEATURES],
                               index=future_idx)
df_future_feats.insert(0, "momento", future_idx)
df_future_feats = pd.concat([df_future_feats.reset_index(drop=True),
                             cyclic_future.reset_index(drop=True)], axis=1)

# ------------------------- 7) exporta a Excel --------------------------------
outfile = "base_con_covs_futuras_2.xlsx"
with pd.ExcelWriter(outfile) as writer:
    df.to_excel(writer, sheet_name="historico", index=False)
    df_future_feats.to_excel(writer, sheet_name="covariables_720h", index=False)
    pd.DataFrame({"mean": y_scaler.mean_, "scale": y_scaler.scale_},
                 index=BASE_FEATURES).to_excel(writer, sheet_name="scalers",
                                               index_label="feature")

print(f"» Archivo Excel generado: {outfile}")


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 672, 9)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_15 (Conv1D)  │ (None, 672, 32)   │        896 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 672, 32)   │          0 │ conv1d_15[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_16 (Conv1D)  │ (None, 672, 32)   │      3,104 │ dropout_14[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 672, 32)   │          0 │ conv1d_16[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_17 (Conv1D)  │ (None, 672, 32)   │        320 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_7 (Add)         │ (None, 672, 32)   │          0 │ dropout_15[0][0], │
│                     │                   │            │ conv1d_17[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_7        │ (None, 672, 32)   │          0 │ add_7[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_18 (Conv1D)  │ (None, 672, 32)   │      3,104 │ activation_7[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_16          │ (None, 672, 32)   │          0 │ conv1d_18[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_19 (Conv1D)  │ (None, 672, 32)   │      3,104 │ dropout_16[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_17          │ (None, 672, 32)   │          0 │ conv1d_19[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_8 (Add)         │ (None, 672, 32)   │          0 │ dropout_17[0][0], │
│                     │                   │            │ activation_7[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_8        │ (None, 672, 32)   │          0 │ add_8[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_20 (Conv1D)  │ (None, 672, 32)   │      3,104 │ activation_8[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_18          │ (None, 672, 32)   │          0 │ conv1d_20[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_21 (Conv1D)  │ (None, 672, 32)   │      3,104 │ dropout_18[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_19          │ (None, 672, 32)   │          0 │ conv1d_21[0][0]   │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_9 (Add)         │ (None, 672, 32)   │          0 │ dropout_19[0][0], │
│                     │                   │            │ activation_8[0][

 Total params: 77,459,568 (295.48 MB)

 Trainable params: 77,459,568 (295.48 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/150
959/959 - 1738s - 2s/step - loss: 0.2047 - val_loss: 0.1277
Epoch 2/150
959/959 - 1696s - 2s/step - loss: 0.1127 - val_loss: 0.1245
Epoch 3/150
959/959 - 1729s - 2s/step - loss: 0.1084 - val_loss: 0.1200
Epoch 4/150
959/959 - 1658s - 2s/step - loss: 0.1063 - val_loss: 0.1280
Epoch 5/150
959/959 - 1742s - 2s/step - loss: 0.1053 - val_loss: 0.1432
Epoch 6/150
959/959 - 1701s - 2s/step - loss: 0.1049 - val_loss: 0.1396
Epoch 7/150
959/959 - 1673s - 2s/step - loss: 0.1032 - val_loss: 0.1403
Epoch 8/150
959/959 - 1655s - 2s/step - loss: 0.1009 - val_loss: 0.1377
Epoch 9/150
959/959 - 1741s - 2s/step - loss: 0.1006 - val_loss: 0.1366
Epoch 10/150
959/959 - 1695s - 2s/step - loss: 0.0984 - val_loss: 0.1451
Epoch 11/150
959/959 - 1698s - 2s/step - loss: 0.0967 - val_loss: 0.1459
Epoch 12/150
959/959 - 1702s - 2s/step - loss: 0.0948 - val_loss: 0.1453
Epoch 13/150
959/959 - 1701s - 2s/step - loss: 0.0937 - val_loss: 0.1650


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 543ms/step


/tmp/ipython-input-6-3535900333.py:124: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  future_idx = pd.date_range(start=CUTOFF + pd.Timedelta(hours=1),


» Archivo Excel generado: base_con_covs_futuras_2.xlsx


In [ ]:
model.save("tcn_covariables.h5")

In [ ]:
# ------------------------- 8) Calcular error para cada variable -------------------------------
# Desescalar las predicciones del conjunto de prueba para comparar con Y_test_raw
pred_test_scaled = model.predict(X_test)
pred_test = y_scaler.inverse_transform(pred_test_scaled.reshape(-1, len(BASE_FEATURES))).reshape(Y_test_raw.shape)

# Calcular el MAE y MSE para cada variable
mae_per_variable = {}
mse_per_variable = {}

for i, var_name in enumerate(BASE_FEATURES):
    # Extraer los valores reales y predichos para la variable actual
    y_true_var = Y_test_raw[:, :, i]
    y_pred_var = pred_test[:, :, i]

    # Calcular el MAE y MSE
    # Flatten los arrays para calcular el error sobre todas las ventanas y horizontes para esta variable
    mae = mean_absolute_error(y_true_var.flatten(), y_pred_var.flatten())
    mse = mean_squared_error(y_true_var.flatten(), y_pred_var.flatten())

    mae_per_variable[var_name] = mae
    mse_per_variable[var_name] = mse

print("\nError por variable (Conjunto de Prueba):")
print("-----------------------------------------")
for var_name in BASE_FEATURES:
    print(f"Variable: {var_name}")
    print(f"  MAE: {mae_per_variable[var_name]:.4f}")
    print(f"  MSE: {mse_per_variable[var_name]:.4f}")
    print("-" * 20)

240/240 ━━━━━━━━━━━━━━━━━━━━ 51s 210ms/step

Error por variable (Conjunto de Prueba):
-----------------------------------------
Variable: dd_Valor
  MAE: 66.1643
  MSE: 7943.7461
--------------------
Variable: VRB_Valor
  MAE: 0.0475
  MSE: 0.0349
--------------------
Variable: Presion_QFE
  MAE: 1.8021
  MSE: 5.1183
--------------------
Variable: Temperatura
  MAE: 2.0350
  MSE: 6.7170
--------------------
Variable: Presion_QFF
  MAE: 1.7424
  MSE: 4.8065
--------------------
